# Recommender System Approaches
This notebooks was written for the purpose of experimenting with different algorithms in recommender systems. 

We reserve the following assumptions for all the algorithms:
- Implicit feedback with binary data. We only know if a user interacted with an item (1), or not (0).
- Data is split using the user-based, time-ordered (can also be random), leave-one-out strategy. This means that for each users we sort their interactions and split into train/val/test in the following way. The last interaction is the test data. The one before is the validation data. The rest is the training data.
- Evaluate on NDCG and Hit Ratio (at 1,3,5,10)

In [1]:
from data.dataset import get_recdataset_dataloader
from utilities.eval import evaluate_recommender_algorithm
#from utilities.utils import reproducible, print_results
#from utilities.eval import Evaluator

import sys
sys.path.append('./')
sys.path.append('./algorithms')

#from scipy import sparse as sp
#from scipy.sparse import linalg as sp_lin
#import scipy as sc
#import numpy as np
#import bottleneck as bn
#from functools import partial

#from tqdm.notebook import tqdm,trange

#from torch import nn 
#from utilities.utils import general_weight_init


In [2]:
train_loader = get_recdataset_dataloader(
                    data_path='./data/lfm2b-1m',
                    split_set='train',
                    n_neg=10,
                    neg_strategy='uniform',
                    batch_size=64,
                    shuffle=True,
                    num_workers=8,
)
val_loader = get_recdataset_dataloader(
                    data_path='./data/lfm2b-1m',
                    split_set='val',
                    n_neg=99,
                    neg_strategy='uniform',
                    batch_size=64,
                    num_workers=8,
)

test_loader = get_recdataset_dataloader(
                    data_path='./data/lfm2b-1m',
                    split_set='test',
                    n_neg=99,
                    neg_strategy='uniform',
                    batch_size=64,
                    num_workers=8,
)

Loading data
Built RecDataset module 
- data_path: ./data/lfm2b-1m 
- n_users: 3555 
- n_items: 77985 
- n_interactions: 870255 
- split_set: train 
- n_neg: 10 
- neg_strategy: uniform 

Loading data
Built RecDataset module 
- data_path: ./data/lfm2b-1m 
- n_users: 3555 
- n_items: 77985 
- n_interactions: 3555 
- split_set: val 
- n_neg: 99 
- neg_strategy: uniform 

Loading data
Built RecDataset module 
- data_path: ./data/lfm2b-1m 
- n_users: 3555 
- n_items: 77985 
- n_interactions: 3555 
- split_set: test 
- n_neg: 99 
- neg_strategy: uniform 



# Random Algorithm
For each user, recommend items sampled u.a.r 

In [3]:
from algorithms.naive_algs import RandomItems

rand_alg = RandomItems()

evaluate_recommender_algorithm(rand_alg,test_loader)

del rand_alg

Built RecommenderAlgorithm module
Built RandomItems module
ndcg@1     : 0.009
hit_ratio@1 : 0.009
ndcg@3     : 0.022
hit_ratio@3 : 0.031
ndcg@5     : 0.029
hit_ratio@5 : 0.048
ndcg@10    : 0.046
hit_ratio@10 : 0.101
ndcg@50    : 0.131
hit_ratio@50 : 0.511


# Popular Items
For each item, assign a score that depends on the popularity of the item

In [4]:
from algorithms.naive_algs import PopularItems

pop_alg = PopularItems(test_loader.dataset.pop_distribution)

evaluate_recommender_algorithm(pop_alg,test_loader)

del pop_alg

Built RecommenderAlgorithm module
Built PopularItems module
ndcg@1     : 0.044
hit_ratio@1 : 0.044
ndcg@3     : 0.083
hit_ratio@3 : 0.111
ndcg@5     : 0.101
hit_ratio@5 : 0.156
ndcg@10    : 0.132
hit_ratio@10 : 0.251
ndcg@50    : 0.205
hit_ratio@50 : 0.586


# SVD (constrainted matrix factorization)

In [5]:
from algorithms.mf_algs import SVDAlgorithm

svd_alg = SVDAlgorithm(factors=100)
svd_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(svd_alg,test_loader)

del svd_alg

Built RecommenderAlgorithm module
Built SVDAlgorithm module 
- factors: 100 
Starting Fitting
End Fitting
ndcg@1     : 0.189
hit_ratio@1 : 0.189
ndcg@3     : 0.277
hit_ratio@3 : 0.340
ndcg@5     : 0.308
hit_ratio@5 : 0.417
ndcg@10    : 0.351
hit_ratio@10 : 0.551
ndcg@50    : 0.422
hit_ratio@50 : 0.869


# User-based KNN (Collaborative Filtering)
There are numerous extensions of KNN. Here we just consider variation in terms of the similarity functions used.
In the end, we generate a n_user x n_user similarity matrix. The prediction for an item i and and user u is given by the weighted average of the similarities of user u and its top-k neighbours. 

In [6]:
from algorithms.knn_algs import UserKNN
from utilities.similarities import SimilarityFunction

#### Jaccard Similarity
Defined a $\frac{|I_x \cap I_y|}{|I_x \cup I_y|}$ where $I_x$ and $I_y$ are the set of items interacted by user x and y respectively. 

**Attempt to a intuitive explanation/reading the formula**:
Two users are considered similar if they have a high number of items that both have consumed. This quantity is normalized by their 'total' items consumed. The last part allows to compare users with different number of items consumed. 

In [7]:
user_jaccard_alg = UserKNN(SimilarityFunction.jaccard,k=100)
user_jaccard_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_jaccard_alg,test_loader)

del user_jaccard_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: <function compute_jaccard_sim_mtx at 0x7fc988d91ee0> 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.225
hit_ratio@1 : 0.225
ndcg@3     : 0.310
hit_ratio@3 : 0.372
ndcg@5     : 0.344
hit_ratio@5 : 0.453
ndcg@10    : 0.380
hit_ratio@10 : 0.566
ndcg@50    : 0.405
hit_ratio@50 : 0.664


### Cosine Similarity
Defined as $\frac{\sum_i u_x^i \cdot u_y^i}{\sqrt{\sum_i {u_x^i}^2} \sqrt{\sum_i {u_y^i}^2}}$ where $u_x$ and $u_y$ are the row vectors of the user-item interaction matrix

**Attempt to a intuitive explanation/reading the formula**: (Note that in the case of binary data as it is now, the cosine similarity is ~similar to jaccard. The difference is in the denominator: while for jaccard you simiply sum the elements in the union, for cosine we sum the square roots of the sum of the elements in each set.). The cosine similarity measures the angle between two vectors. Assuming that each item represent a specific dimension, the cosine ~checks how these vectors are oriented in the multidimensional space. If the vectors are oriented in the same direction (regardless of their intensity/length), then they are similar. If they are oriented in opposite directions, then they are dissimilar.


In [8]:
user_cosine_alg = UserKNN(SimilarityFunction.cosine,k=100)
user_cosine_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_cosine_alg,test_loader)

del user_cosine_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: <function compute_cosine_sim_mtx at 0x7fc988d91940> 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.228
hit_ratio@1 : 0.228
ndcg@3     : 0.322
hit_ratio@3 : 0.391
ndcg@5     : 0.355
hit_ratio@5 : 0.472
ndcg@10    : 0.393
hit_ratio@10 : 0.587
ndcg@50    : 0.434
hit_ratio@50 : 0.760


### Pearson Correlation
Defined as $\frac{\sum_i (u_x^i - \bar{u_x}) \cdot (u_y^i - \bar{u_y})}{\sqrt{\sum_i {(u_x^i - \bar{u_x})}^2} \sqrt{\sum_i {(u_y^i - \bar{u_y})}^2}}$ where $u_x$ and $u_y$ are the row vectors of the user-item interaction matrix and $\bar{u_x}$ and $\bar{u_y}$ the respective means

**Attempt to a intuitive explanation/reading the formula**: In general, pearson correlation evaluates if two measurments are linearly dependent to each other (i.e. if you can write Y=aX + b where X,Y are user vector ratings and a,b are parameters). The definition is similar to the cosine similarity but by subtracting the mean of the measurments , computed over *all* the items. Note that, otherwise, the mean would be 1, and thus leading to 0 everywhere when removing the mean (1-1=0). This mean, then, should take into account some information about the number consumed by the users (the higher the number of items consumed, the bigger the mean), similarly what the denominator does for the jaccard similarity. The mean is then removed by the vectors (only non-zero entries) and the definition of cosine similarity is applied. The mean is removed only to these entries because... performance? Removing the mean to 0s makes them go below 0. Does it make sense at all to have a negative similarity? Looking at the results, it seems that the metrics  are even better for the dense definition below



In [9]:
user_pearson_alg = UserKNN(SimilarityFunction.pearson,k=100)
user_pearson_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_pearson_alg,test_loader)

del user_pearson_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: <function compute_pearson_sim_mtx at 0x7fc988d918b0> 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.228
hit_ratio@1 : 0.228
ndcg@3     : 0.322
hit_ratio@3 : 0.391
ndcg@5     : 0.355
hit_ratio@5 : 0.472
ndcg@10    : 0.393
hit_ratio@10 : 0.587
ndcg@50    : 0.434
hit_ratio@50 : 0.760


### Asymmetric Cosine Similarity
Defined as $\frac{|I_x \cap I_y|}{|I_x|^\alpha |I_y|^{1-\alpha}}$ where $I_x$ and $I_y$ are the set of items interacted by user x and y respectively and $\alpha$ is a hyperparameter. (see also [here](https://dl.acm.org/doi/pdf/10.1145/2507157.2507189))

**Attempt to a intuitive explanation/reading the formula**: The cosine similarity can be generalized in such a way to give different weght to the different cardilnalities of the  item sets. The paper reports this example for the items. Imagine you are comparing two songs and you want to estimates their similarity. Assume that song A is a popular song from (changing the band) Muse e.g. Madness and song B a more unknown song by them e.g. Unintended. If we know that a user consumed B, it is likely that the user is a Muse's fan and will also consume A. If we know that user consumed A, it is not clear if the user just listened to this popular song or is a fan of Muse (and consume song B). This leads to: song B is more similar to song A than song A is to B. For this reason, we scale the weight given a specific alpha. As an example, consider that Song A got 100 users, Song B 12 users and only 10 users listened to both Song A and B

In [10]:
user_asymmetric_cosine_alg = UserKNN(SimilarityFunction.asymmetric_cosine,k=100,alpha=0.6)
user_asymmetric_cosine_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_asymmetric_cosine_alg,test_loader)

del user_asymmetric_cosine_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: functools.partial(<function compute_asymmetric_cosine_sim_mtx at 0x7fc988d91670>, 0.6) 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.219
hit_ratio@1 : 0.219
ndcg@3     : 0.314
hit_ratio@3 : 0.385
ndcg@5     : 0.348
hit_ratio@5 : 0.466
ndcg@10    : 0.386
hit_ratio@10 : 0.584
ndcg@50    : 0.431
hit_ratio@50 : 0.775


### Sørensen–Dice Coefficient
Defined as $2\frac{|I_x \cap I_y|}{|I_x| + |I_y|}$ where $I_x$ and $I_y$ are the set of items interacted by user x and y respectively. (see also [here](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient))

**Attempt to a intuitive explanation/reading the formula**:
No real explanation for this one. It basically just give twice the weight to the shared information. (It is also very similar to Jaccard)

In [11]:
user_sorensen_dice_alg = UserKNN(SimilarityFunction.sorensen_dice,k=100)
user_sorensen_dice_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_sorensen_dice_alg,test_loader)

del user_sorensen_dice_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: <function compute_sorensen_dice_sim_mtx at 0x7fc988d91ca0> 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.228
hit_ratio@1 : 0.228
ndcg@3     : 0.321
hit_ratio@3 : 0.389
ndcg@5     : 0.358
hit_ratio@5 : 0.478
ndcg@10    : 0.394
hit_ratio@10 : 0.590
ndcg@50    : 0.434
hit_ratio@50 : 0.754


### Tversky Index
Defined as $\frac{|I_x \cap I_y|}{|I_x \cap I_y| + \alpha|I_x - I_y| + \beta|I_y - I_x|}$ where $I_x$ and $I_y$ are the set of items interacted by user x and y respectively and $\alpha$ and $\beta$ are hyperparameters. (see also [here](https://en.wikipedia.org/wiki/Tversky_index))

**Attempt to a intuitive explanation/reading the formula**:
As Asymmetric Cosine, the Tversky index is an asymmetric similarity function. It basically gives a 'different' weighting scheme for the common/ not-in-common items compared to asymmetric cosine. 

In [13]:
user_tversky_alg = UserKNN(SimilarityFunction.tversky,k=100,alpha=0.8,beta=0.6)
user_tversky_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(user_tversky_alg,test_loader)

del user_tversky_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: functools.partial(<function compute_tversky_sim_mtx at 0x7fc988d91c10>, 0.8, 0.6) 
- k: 100 

Built UserKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.226
hit_ratio@1 : 0.226
ndcg@3     : 0.320
hit_ratio@3 : 0.387
ndcg@5     : 0.357
hit_ratio@5 : 0.478
ndcg@10    : 0.392
hit_ratio@10 : 0.588
ndcg@50    : 0.433
hit_ratio@50 : 0.758


# Item-based KNN (Collaborative Filtering)
We generate a n_items x n_items similarity matrix. The prediction for an item i and and user u is given by the weighted average of the similarities of item i and its top-k neighbours that also were consumed by user u. 

**Most of the similarityfunctions should be further optimized in order to work for huge similarity matrices!! It is already a miracle if it works with the cosine!**

### Cosine Similarity

In [16]:
from algorithms.knn_algs import ItemKNN
from utilities.similarities import SimilarityFunction

item_cosine_alg = ItemKNN(SimilarityFunction.cosine,k=100)
item_cosine_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(item_cosine_alg,test_loader)

del item_cosine_alg

Built RecommenderAlgorithm module
Built KNNAlgorithm module 
- sim_func: <function compute_cosine_sim_mtx at 0x7fc988d91940> 
- k: 100 

Built ItemKNN module 

Starting Fitting
End Fitting
ndcg@1     : 0.352
hit_ratio@1 : 0.352
ndcg@3     : 0.458
hit_ratio@3 : 0.532
ndcg@5     : 0.485
hit_ratio@5 : 0.597
ndcg@10    : 0.508
hit_ratio@10 : 0.669
ndcg@50    : 0.516
hit_ratio@50 : 0.702


# SLIM

In [17]:
from algorithms.linear_algs import SLIM

slim_alg = SLIM(alpha=1e-1,l1_ratio=1e-2,max_iter=100)
slim_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(slim_alg,test_loader)

del slim_alg

Built RecommenderAlgorithm module
Built SLIM module 
- alpha: 0.1 
- l1_ratio: 0.01 
- max_iter: 100 

Running on 8 cores


48740 -> 58488:   0%|                       | 22/9748 [00:12<1:25:24,  1.90it/s]

KeyboardInterrupt: 

# EASE

In [ ]:
# EASE would implode with this amount of items

# Deep Matrix Factorization
[paper](https://www.ijcai.org/Proceedings/2017/0447.pdf)

In [6]:
from algorithms.neural_alg import DeepMatrixFactorization
from utilities.trainer import ExperimentConfig,Trainer


dmf_alg = DeepMatrixFactorization(train_loader.dataset.csr_matrix,u_mid_layers=[100],i_mid_layers=[100],final_dimension=32)
conf = ExperimentConfig(lr=1e-4,device='cpu')
trainer = Trainer(dmf_alg,train_loader,val_loader,conf)

RecommenderAlgorithm class crated
SGDBasedRecommenderAlgorithm class crated
DeepMatrixFactorization class created
Built Trainer module


In [9]:
trainer.fit()

Validation started
Init - Avg Val Value 0.090 



KeyboardInterrupt: 

# $\text{P}^3_\alpha$

In [3]:
from graph_algs import P3alpha
p3_alg = P3alpha()
p3_alg.fit(train_loader.dataset.csr_matrix)

evaluate_recommender_algorithm(p3_alg,test_loader)

Built RecommenderAlgorithm module
Built P3alpha module
Start fitting
Frist Step done
Second Step done
End fitting
ndcg@1     : 0.233
hit_ratio@1 : 0.233
ndcg@3     : 0.337
hit_ratio@3 : 0.412
ndcg@5     : 0.378
hit_ratio@5 : 0.512
ndcg@10    : 0.425
hit_ratio@10 : 0.657
ndcg@50    : 0.486
hit_ratio@50 : 0.927


# Alternating Least Square

In [9]:
from mf_algs import AlternatingLeastSquare
als_alg = AlternatingLeastSquare(1,64,1e-2,500)
als_alg.fit(train_loader.dataset.csr_matrix,False)

evaluate_recommender_algorithm(als_alg,test_loader)

Built RecommenderAlgorithm module
Built AlternatingLeastSquare module 
- alpha: 1 
- factors: 64 
- regularization: 0.01 
- n_iterations: 500 

Starting Fitting


  0%|          | 0/500 [00:00<?, ?it/s]

End Fitting
ndcg@1     : 0.170
hit_ratio@1 : 0.170
ndcg@3     : 0.258
hit_ratio@3 : 0.322
ndcg@5     : 0.290
hit_ratio@5 : 0.403
ndcg@10    : 0.334
hit_ratio@10 : 0.538
ndcg@50    : 0.410
hit_ratio@50 : 0.879


In [3]:
from neural_alg import UProtoMF,IProtoMF,UIProtoMF
from utilities.trainer import ExperimentConfig,Trainer


ui_protomf = UIProtoMF(train_loader.dataset.n_users,train_loader.dataset.n_items,latent_dimension=32)
conf = ExperimentConfig(lr=1e-4,device='cpu')
trainer = Trainer(ui_protomf,train_loader,val_loader,conf)

Built RecommenderAlgorithm module
Built SGDBasedRecommenderAlgorithm module
Built UIProtoMF model 
- n_users: 3555 
- n_items: 77985 
- latent_dimension: 32 
- u_n_prototypes: 20 
- i_n_prototypes: 20 
- u_sim_proto_weight: 1.0 
- u_sim_batch_weight: 1.0 
- i_sim_proto_weight: 1.0 
- i_sim_batch_weight: 1.0 

Built Trainer module 
- n_epochs: 100 
- rec_loss: RecBinaryCrossEntropy 
- device: cpu 
- optimizing_metric: hit_ratio@10 
- max_patience: 10 
- best_model_path: ../models/UIProtoMF_best_model.pth 



In [4]:
trainer.fit()

Validation started
Init - Avg Val Value 0.103 



  0%|                                                   | 0/100 [00:06<?, ?it/s]


KeyboardInterrupt: 